# Property Map Collective Variable Force Field Correction Pipeline
---
The pipeline generates force field corrections in .pdb format and is divided into these steps:

1. [Molecule shape processing](#1.-Molecule-shape-processing)
2. [Preparation of environment and molecule](#2.-Preparation-of-environment-and-molecule)
3. [Generation of representative configurations](#3.-Generation-of-representative-configurations)
4. [Accurate and innacurate energy evaluation](#4.-Accurate-and-innacurate-energy-evaluation)
5. [Define correction of force field](#5.-Define-correction-of-force-field)

In [ ]:
import os
import re
import sys
import math
import time
import shutil
import subprocess

# import chemical software
from rdkit.Chem.Draw import IPythonConsole
from rdkit import Chem
from tqdm.notebook import tqdm
from molvs import Standardizer
import nglview as nv
import pytraj as pt
import matplotlib.pyplot as plt

# import custom libraries
from draw_3d import drawit
from modules.convert import convert_to_orca_methods
from modules.plot_graph import plot_landmarks
from modules.gmx_orca.gmx_orca_run import gmx_run, orca_run, parallel_wait
from modules.replace import replace_text_for_embedding

# path to needed scripts
orca_job_check = '/home/base/modules/orcajobcheck.py'

## 1. Molecule shape processing

In [ ]:
# specify input of desired molecule in SMILES
smiles_molecule = 'CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)C[NH+]3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5'

# set visualization parameters
# ... add Indices to molecule image
IPythonConsole.drawOptions.addAtomIndices = True

# ... set molecule size
IPythonConsole.molSize = 900,900


molecule = Chem.MolFromSmiles(smiles_molecule)
molecule

In [ ]:
s = Standardizer()
molecule = s.standardize(molecule)
molecule = Chem.AddHs(molecule)
natoms = molecule.GetNumAtoms()
charge = Chem.rdmolops.GetFormalCharge(molecule)

molecule

### 1.1 Set the lowest energy configuration
Perform basic energy minimisation by running [Merck molecular force field (MMFF94)](https://open-babel.readthedocs.io/en/latest/Forcefields/mmff94.html) and choose the conformation with the lowest energy.

Visualize the configuration with lowest energy in 3D afterwards.

In [ ]:
# number of configurations to generate
numc = 50

Chem.AllChem.EmbedMultipleConfs(molecule, clearConfs=True, numConfs=numc)

# run MMFF94
optim = Chem.AllChem.MMFFOptimizeMoleculeConfs(molecule)

minid = -1
minene = sys.float_info.max
for i in range(len(optim)):
    if optim[i][1] < minene:
        minene = optim[i][1]
        minid = i

# write to file molekula.mol for further processing
writer = Chem.SDWriter('molekula.mol')
writer.write(molecule, confId = minid)

drawit(molecule, confId = minid)

### 1.2 Detect torsion angles
Detect torsions based on pattern in *smarts*. Output can be checked on 2D visualization in step [2. Molecule shape processing](#2.-Molecule-shape-processing).

In [ ]:
bond_smarts = '''[!$([NH]!@C(=O))&!D1&!$(*#*)&!$([C;H3])&!$([O;H1])&!$([N;H3])]-&!@
                 [!$([NH]!@C(=O))&!D1&!$(*#*)&!$([C;H3])&!$([O;H1])&!$([N;H3])]'''

rotatable_bond = Chem.MolFromSmarts(bond_smarts)
rotatables = molecule.GetSubstructMatches(rotatable_bond)
print(f'Rotatables: {rotatables}')


torsions = []
for rotatable in rotatables:
    pairs1 = []
    pairs2 = []
    for bond in molecule.GetBonds():
        if rotatable[0] == bond.GetBeginAtomIdx() and rotatable[1] != bond.GetEndAtomIdx():
            pairs1.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
        if rotatable[1] == bond.GetBeginAtomIdx() and rotatable[0] != bond.GetEndAtomIdx():
            pairs2.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    torsions.append([pairs1[0][1], pairs1[0][0], pairs2[0][0], pairs2[0][1]])
print(f'Torsions: {torsions}')

## 2. Preparation of environment and molecule
### 2.1 Perform energetic minimisation
Generate config file and other files needed for energetic minimisation. During generation an ordering of atoms will get wrong. This is fixed afterwards from output files to fit the minimised molecule.

Finally perform an energetic minimisation using [Gromacs](https://www.gromacs.org/). This pipeline uses wrapper so the Gromacs can be run independently to this environment. Please read the wrapper [documentation](https://github.com/CERIT-SC/pmcvff-correction/tree/jupyter-refactor/modules/gmx_orca) before you interact with any Gromacs command.

In [ ]:
with open("em/em.mdp", "w") as emfile:
    emfile.write("integrator          =  steep\n")
    emfile.write("nsteps              =  100000\n")
    emfile.write("emtol               =  0\n")
    emfile.write("emstep              =  0.1\n")
    emfile.write("nstcomm             =  1\n")
    emfile.write("nstxout             =  100\n")
    emfile.write("nstvout             =  100\n")
    emfile.write("nstfout             =  0\n")
    emfile.write("nstlog              =  100\n")
    emfile.write("nstenergy           =  100\n")
    emfile.write("nstlist             =  1\n")
    emfile.write("ns_type             =  grid\n")
    emfile.write("coulombtype         =  cut-off\n")
    emfile.write("rlist               =  1.4\n")
    emfile.write("rcoulomb            =  1.4\n")
    emfile.write("rvdw                =  1.4\n")
    emfile.write("energygrps          =  System\n")
    emfile.write("epsilon-r           =  80\n")
    emfile.write("\n")
shutil.copy("MOL_GMX.gro", "em/")
shutil.copy("MOL_GMX.top", "em/")


!antechamber -i molekula.mol -fi mdl -o molekula.prepi -fo prepi -c bcc -nc {charge} && \
parmchk2 -i molekula.prepi -f prepi -o molekula.frcmod && \
tleap -f tleapin.txt && \
acpype -p molekula.prmtop -x molekula.inpcrd

In [ ]:
# fix ordering of atoms

order_before = []
with open('sqm.pdb','r') as pdbfile:
    for atom in pdbfile.readlines():
        order_before.append(atom.split()[2])
        
order_after = []
with open('MOL_GMX.gro','r') as grofile:
    for atom in grofile.readlines():
        if atom.startswith('    1  MOL'):
            order_after.append(atom.split()[2])

            
torsions_new = []
torsion_new = []
for torsion in torsions:
    for i in torsion:       
        torsion_new.append(order_after.index(order_before[i])+1)
    torsions_new.append(torsion_new)
    torsion_new = []
    
torsions = torsions_new
print(f'New torsions: {torsions}')

In [ ]:
gmx_run('editconf -f MOL_GMX -o box -c -box 3 3 3', workdir='em')
gmx_run('grompp -f em.mdp -c box -p MOL_GMX -o em1', workdir='em')
gmx_run('mdrun -deffnm em1 -ntomp 2', workdir='em')

### 2.2 Perform molecular dynamics simulation
Create config file and perform molecular dynamics simulation. Simulation trajectory can be visualized.

Afterwards a [periodic boundary conditions](https://www.gromacs.org/Documentation_of_outdated_versions/Terminology/Periodic_Boundary_Conditions) must be applied so the molecule "does not jump out of the box". 

In [ ]:
with open('md/md.mdp', 'w') as mdfile:
    mdfile.write('integrator          = sd\n')
    mdfile.write('nsteps              = 100000\n')
    mdfile.write('dt                  = 0.001\n')
    mdfile.write('nstxout             = 1000\n')
    mdfile.write('nstvout             = 1000\n')
    mdfile.write('nstenergy           = 1000\n')
    mdfile.write('nstlog              = 1000\n')
    mdfile.write('continuation        = no\n')
    mdfile.write('constraints         = none\n')
    mdfile.write('cutoff-scheme       = Verlet\n')
    mdfile.write('ns_type             = grid\n')
    mdfile.write('nstlist             = 1\n')
    mdfile.write('rlist               = 1.4\n')
    mdfile.write('rcoulomb            = 1.4\n')
    mdfile.write('rvdw                = 1.4\n')
    mdfile.write('coulombtype         = cut-off\n')
    mdfile.write('tcoupl              = V-rescale\n')
    mdfile.write('tc-grps             = system\n')
    mdfile.write('tau_t               = 0.1\n')
    mdfile.write('ref_t               = 300\n')
    mdfile.write('pcoupl              = no\n')
    mdfile.write('pbc                 = xyz\n')
    mdfile.write('gen_vel             = yes\n')
    mdfile.write('epsilon-r           = 80\n')
    mdfile.write('\n')
shutil.copy('em/em1.gro', 'md/')
shutil.copy('MOL_GMX.top', 'md/')

In [ ]:
gmx_run('grompp -f md.mdp -c em1 -p MOL_GMX -o md1', workdir='md')
gmx_run('mdrun -deffnm md1 -ntomp 2', workdir='md')

In [ ]:
# convert trajectory to .pdb format so it can be visualized

# select group for trjconv evaluation output
# Group     0 (         System)
# Group     1 (          Other)
# Group     2 (            MOL)
group = '0'


gmx_run('trjconv -pbc nojump -s md1.tpr -f md1.trr -o outTraj.pdb', workdir='md', groups=group)

In [ ]:
# visualize the molecular dynamics trajectory
traj = pt.load('md/outTraj.pdb')
view = nv.show_pytraj(traj)
view

In [ ]:
# fix periodic boundaries errors 

# select group for trjconv evaluation output
# Group     0 (         System)
# Group     1 (          Other)
# Group     2 (            MOL)
group = '1'


for i in tqdm(range(len(torsions))):
    fr = str(float(100-len(torsions)+i)-0.01)
    to = str(float(100-len(torsions)+i)+0.01)
    gmx_run(f'trjconv -pbc nojump -s md1 -f md1 -o frame{i}.gro -b {fr} -e {to}<<EOF\n0\nEOF',
            workdir='md',
            groups=group)

## 3. Generation of representative configurations
### 3.1 Trajectory generation
Create config file and *plumed.dat* file. Based on these files run the simulation (with metadynamics) to generate a trajectory.

[Plumed](https://www.plumed.org/) is used to run metadynamics.

In [ ]:
with open('mtd/mtd.mdp', 'w') as mtdfile:
    mtdfile.write('integrator          = sd\n')
    mtdfile.write('nsteps              = 10000000\n')
    mtdfile.write('dt                  = 0.001\n')
    mtdfile.write('nstxout             = 10000\n')
    mtdfile.write('nstvout             = 10000\n')
    mtdfile.write('nstenergy           = 1000\n')
    mtdfile.write('nstlog              = 1000\n')
    mtdfile.write('continuation        = no\n')
    mtdfile.write('constraints         = none\n')
    mtdfile.write('cutoff-scheme       = Verlet\n')
    mtdfile.write('ns_type             = grid\n')
    mtdfile.write('nstlist             = 1\n')
    mtdfile.write('rlist               = 1.4\n')
    mtdfile.write('rcoulomb            = 1.4\n')
    mtdfile.write('rvdw                = 1.4\n')
    mtdfile.write('coulombtype         = cut-off\n')
    mtdfile.write('tcoupl              = V-rescale\n')
    mtdfile.write('tc-grps             = system\n')
    mtdfile.write('tau_t               = 0.1\n')
    mtdfile.write('ref_t               = 300\n')
    mtdfile.write('pcoupl              = no\n')
    mtdfile.write('pbc                 = xyz\n')
    mtdfile.write('gen_vel             = yes\n')
    mtdfile.write('epsilon-r           = 80\n')
    mtdfile.write('\n')

In [ ]:
for i in range(len(torsions)):
    if not os.path.exists(f'mtd/w{i}'):
        os.mkdir(f'mtd/w{i}'')
        
    with open(f'mtd/w{i}/plumed.dat', "w") as plumeddat:
        plumeddat.write('RANDOM_EXCHANGES\n')
        plumeddat.write(f'WHOLEMOLECULES ENTITY0=1-{natoms}\n')
        for j in range(len(torsions)):
            plumeddat.write(f'''TORSION ATOMS={torsions[j][0]},{torsions[j][1]},
                                              {torsions[j][2]},{torsions[j][3]} LABEL=cv{j+1}\n''')
        plumeddat.write(f'''METAD ARG=cv{i+1} HEIGHT=0.5 SIGMA=0.3 PACE=1000 GRID_MIN=-pi 
                            GRID_MAX=pi BIASFACTOR=15 LABEL=be\n''')
        cvs = ""
        for j in range(len(torsions)):
            cvs = cvs + f'cv{j+1},'
        cvs = cvs[:-1]
        plumeddat.write(f'PRINT ARG={cvs} STRIDE=1000 FILE=COLVAR\n')
        plumeddat.write('PRINT ARG=be.bias STRIDE=1000 FILE=BIAS\n')

In [ ]:
shutil.copy('MOL_GMX.top', 'mtd/')

# perform preprocessing before generation of the trajectory
for i in tqdm(range(len(torsions))):
    shutil.copy(f'md/frame{i}.gro', f'mtd/w{i}/')
    gmx_run(f'grompp -f mtd.mdp -c w{i}/frame{i} -p MOL_GMX -o w{i}/mtd1',
            workdir='mtd',
            parallel=True)
parallel_wait()

In [ ]:
directories = ''
for i in range(len(torsions)):
    directories = directories + f'w{i} '

# see mdrunlog in mtd directory for insight into running 
# mdrun simulation (e.g 'tail -f mdrunlog')
gmx_run(f'mdrun -g mdrunlog -ntomp 1 -deffnm mtd1 -replex 500 -plumed plumed.dat -multidir {directories}', 
        workdir='mtd', 
        mpi_run=len(torsions))

### 3.2 Configurations clustering
Concatinate all the trajectories that simulation produced. Then cluster this trajectory to groups for which one representative configuration is chosen (*cutoff* can be modified for more/less clusters).

Result of [Gromacs clustering](https://manual.gromacs.org/documentation/current/onlinehelp/gmx-cluster.html) is .pdb file containing all representative configurations. These must be divided into separate .pdb files for further processing.

In [ ]:
trajectories = ''
for i in range(len(torsions)):
    trajectories = trajectories + f'mtd/w{i}/mtd1.trr '

# concatinate trajectories    
gmx_run(f'trjcat -f {trajectories} -cat -o mtd/mtd1.trr')

# make index file with non-hydrogen atoms
gmx_run("make_ndx -f md/md1.tpr -o mtd/index.ndx", make_ndx="1&!aH*")


# select groups for cluster evaluation output
# Group     0 (         System)
# Group     1 (          Other)
# Group     2 (            MOL)
# Group     3 (         Custom)
groups = '30'

gmx_run(f'''cluster -method gromos -f mtd/mtd1.trr -s mtd/w0/mtd1.tpr -n mtd/index.ndx -cutoff 0.15 
        -cl clustering/outClusters.pdb''', groups=groups)

In [ ]:
# divide all clusters from clustering output file 
# to single files and index them from 0.
# Also fix missing element of each ATOM on 
# line 77 (by pdb format specification)
cluster_index = 0
i = 0

with open('clustering/outCluster.pdb') as infile:
    clusters = infile.readlines()
    while i < len(clusters):
        with open(f'clustering/outClustersPDB/outCluster{cluster_index}.pdb', 'w') as outfile:
            for line in clusters[i:]:
                split_line = line.split()
                if split_line[0] == 'ATOM':
                    line = line[:77] + split_line[2][0] + '\n'
                out.write(line)
                i += 1
                if line == 'ENDMDL\n':
                    break
            cluster_index += 1

### 3.3 Visualize landmarks
Goal of this part is to compute embeddings which are visualized afterwards. Each step is performed on the trajectory which results from previous step. Base trajectory used in 1st step is the concatinated trajectory from metadynamics simulation.

1. Apply periodic boundary conditions to metadynamics trajectory
2. Perform fitting on the trajectory
3. Remove Hydrogen
4. Train [parmtSNEcv](https://gitlab.ics.muni.cz/spiwokv/parmtSNEcv)
5. Compute embeddings

Finally visualize all generated configurations from metadynamics trajectory in contrast to representative clusters configurations

In [ ]:
# Group     0 (         System)
# Group     1 (          Other)
# Group     2 (            MOL)
# Group     3 (         Custom)
# select group for periodic boundaries check output:
group = '0'
gmx_run('trjconv -f mtd/mtd1.trr -s mtd/w0/mtd1.tpr -pbc mol -o visualization/traj/mtd1_nopbc.xtc', groups=group)

# select groups for fitting and output:
groups = '00'
gmx_run('''trjconv -f visualization/traj/mtd1_nopbc.xtc -s clustering/outClustersPDB/outCluster0.pdb 
        -fit rot+trans -o visualization/traj/mtd1_fit.xtc''', groups=groups)

# select group for no Hydrogen output:
group = '3'
gmx_run('trjconv -f visualization/traj/mtd1_fit.xtc -n mtd/index.ndx -o visualization/traj/mtd1_fit_noH.xtc',
        groups=group)

# select groups for size of the box and output:
groups = '33'
gmx_run('''editconf -f clustering/outClustersPDB/outCluster0.pdb -n mtd/index.ndx -box 3 3 3 -c
        -o visualization/ref.pdb''', groups=groups)


# train parmtSNEcv
!parmtSNEcv -i visualization/traj/mtd1_fit_noH.xtc -p visualization/ref.pdb -boxx 3 -boxy 3 -boxz 3 -dim 2 \
 -layers 2 -o visualization/out.txt -plumed visualization/plumed.dat -epochs 2000


# modify plumed.dat to compute embedding in every step and change name of file for *** mtd trajectory ***
replace_text_for_embedding('visualization/plumed.dat', stride=(100, 1), file_name=('COLVAR', '2d_embedding'))

# fix weights of atoms
replace_text_for_embedding('visualization/ref.pdb', weight=(0.00, 1.00))

# run as plumed
gmx_run('driver --plumed visualization/plumed.dat --mf_xtc visualization/traj/mtd1_fit.xtc')

# modify plumed.dat to compute embedding in every step and change name of file for *** representatives ***
replace_text_for_embedding('visualization/plumed.dat', file_name=('2d_embedding', 'landmarks'))

# run as plumed
gmx_run('driver --plumed visualization/plumed.dat --mf_pdb clustering/outClusters.pdb')

In [ ]:
# visualize configurations in contrast to representative configurations

x = []
y = []
x1 = []
y1 = []
with open("2d_embedding", "r") as infile:
    for line in infile.readlines()[1:]:
        split_values = line.split()
        x.append(split_values[1])
        y.append(split_values[2])
with open("landmarks", "r") as infile:
    for line in infile.readlines()[1:]:
        split_values = line.split()
        x1.append(split_values[1])
        y1.append(split_values[2])
        
        
plt.figure(figsize=(100, 50), dpi=100)
plt.scatter(x, y, c='b', marker='.', label='All Configurations', s=2000)
plt.scatter(x1, y1, c='r', marker='o', label='Representative configurations', s=3000)
plt.legend(loc='upper left', prop={'size': 80})

plt.show()

## 4. Accurate and innacurate energy evaluation
Accurate energy values are computed in this step using the [Orca](https://sites.google.com/site/orcainputlibrary/) quantum chemistry software. Orca uses [method files](https://sites.google.com/site/orcainputlibrary/generalinput) in exact format to perform the computations. Computation of exact energy values of each representative is composed of 3-step chain. Input for *AM1* are representative configurations defined by clustering in previous step. Each next computation is performed on the output of the previous step.

1. **AM1 optimisation** (*input*: clustering results)
2. **BP86 SVP** (*input*: AM1 output)
3. **BP86 TZVP** (*input*: BP86 SVP output, *output*: exact energy values of representative configurations)

Also each step above consists of 3 substeps:

1. Convert the output of previous step to Orca compatible *.inp method*
2. Perform Orca computation
3. Analyse output (see logs)

Please read the wrapper [documentation](https://github.com/CERIT-SC/pmcvff-correction/tree/jupyter-refactor/modules/gmx_orca) before interacting with any Orca command.

In [ ]:
# fix indexing of atoms because Orca indexes from zero 
orca_torsions = [[] for _ in range(len(torsions))]
torsion_index = 0
for torsion in torsions:
    for atom_index in torsion:
        orca_torsions[torsion_index].append(atom_index - 1)
    torsion_index += 1
torsions = orca_torsions

In [ ]:
# initialize a list of all clusters that are considered
# in the quantum chemistry computations. For example
# non-converged simulation on cluster will result in
# discarding that cluster from further computations
#
# ** don't forget to reset clusters variable when
# rerunning computations as non-converged will be
# missing **
clusters = []
for cluster in os.listdir('clustering/outClustersPDB'):
    if '.pdb' in cluster:
        clusters.append(cluster.replace('.pdb', ''))
        

# convert .pdb file to orca method
# - method specifies which method to apply by Orca
# - info is to specify the *charge* and *spin* of molecule
# - nprocs is to specify the number of CPU's to use (None for default)
# - xyz specify True to convert .xyz to orca instead
def pdb2orca(pdb_in, orca_out, method, info, nprocs=None, xyz=False):
    with open(pdb_in, 'r') as infile, open(orca_out, 'w') as outfile:
        outfile.write(f'{method}\n')
        
        if nprocs:
            outfile.write(f'%pal\n')
            outfile.write(f'nprocs {str(nprocs)}\n')
            outfile.write(f'end\n')
            
        outfile.write('%geom\n')
        outfile.write('Constraints\n')
        
        # write torsions
        for torsions_list in torsions:
            outfile.write('{D ')
            for torsion in torsions_list:
                outfile.write(f'{str(torsion)} ')
            outfile.write('C}\n')
        
        outfile.write('end\n')
        outfile.write('end\n\n')
        outfile.write(f'*xyz {info[0]} {info[1]}\n')
        
        # copy atom information from input and modify
        # to fit the orca method format
        if xyz:
            for line in infile.readlines()[2:]:
                outfile.write(f'{line}')
        else:
            for line in infile.readlines():
                splitline = line.split()
                if splitline[0] == 'ATOM':
                    orca_line = f'{splitline[10]}       {splitline[5]}      {splitline[6]}       {splitline[7]}\n'
                    outfile.write(orca_line)
                
        outfile.write('*\n')

### 4.2 AM1 geometry optimisation method
Perform the semi-empirical [Austin Model 1](https://en.wikipedia.org/wiki/Austin_Model_1) method on representatives. It can be understood as kind of preprocessing (optimisation) which uses approximations instead of exact calculations to speed up the process of the next simulation. Output of AM1 is input for the next [BP86 SVP method.](###-4.3-BP86-SVP-method).

In [ ]:
# convert the representatives from clustering to fit 
# the AM1 Orca method

# specify AM1 method
method = '!AM1 Opt'

# input directory of .pdb representatives
input_dir = 'clustering/outClustersPDB/'

# output directory where converted .inp Orca 
# methods will be placed
output_dir = 'am1/input/'

# specify spin (and charge)
spin = 1
# charge is specified in the first step 
# of 1. Molecule shape processing
# charge = XXX


for pdb in tqdm(os.listdir(input_dir)):
    if '.pdb' in pdb:
        infile = input_dir + pdb
        outfile = output_dir + pdb.replace('pdb', 'inp')

        pdb2orca(infile, outfile, method, (charge,spin))

In [ ]:
# run the AM1 method optimisation

input_path = "am1/input/"
output_path = "am1/output/"


for method_file in os.listdir(input_path):
    if '.inp' in method_file:
        log_file = method_file.replace('inp', 'out')
        cluster_dir = method_file.replace('.inp', '/')
        cluster_dir_path = output_path + cluster_dir

        if not os.path.exists(cluster_dir_path):
            os.mkdir(cluster_dir_path)    
        shutil.copy(input_path + method_file, cluster_dir_path)

        #temporary log !!!!!!!!!!!!!
        orca_run(method_file, cluster_dir_path + log_file, workdir=cluster_dir_path, parallel=True)
    
parallel_wait()

In [ ]:
# check the output of AM1 optimisation

for cluster in clusters:
    with open(f'{output_path}{cluster}/{cluster}.out') as infile:
        orca_log = infile.read()
        if '****ORCA TERMINATED NORMALLY****' not in orca_log:
            print(orca_log)
            raise SystemExit(f'Error in AM1 method of {cluster}. You should not continue computation!')
print(f'AM1 has been successfully finished on all clusters. You can view logs at "{output_path}"')

### 4.3 BP86 SVP General gradient approximation method

In [ ]:
# specify BP86 method
method = '!BP86 def2-SVP TightSCF Opt'

# input directory of .pdb representatives
input_dir = 'am1/output/'

# output directory where converted .inp Orca 
# methods will be placed
output_dir = 'bp86svp/input/'

# number of CPUs to use
nprocs = 12

# specify spin (and charge)
spin = 1
# charge is specified in the first step 
# of 1. Molecule shape processing
# charge = XXX


for cluster in clusters:
    infile = f'{input_dir}{cluster}/{cluster}.xyz'
    outfile = f'{output_dir}{cluster}.inp'
    
    pdb2orca(infile, outfile, method, (charge,spin), nprocs=nprocs, xyz=True)

In [ ]:
# run the BP86 General gradient approximation method

input_path = 'bp86svp/input/'
output_path = 'bp86svp/output/'


for method_file in os.listdir(input_path):
    if '.inp' in method_file:
        log_file = method_file.replace('inp', 'out')
        cluster_dir = method_file.replace('.inp', '/')
        cluster_dir_path = output_path + cluster_dir

        if not os.path.exists(cluster_dir_path):
            os.mkdir(cluster_dir_path)    
        shutil.copy(input_path + method_file, cluster_dir_path)

        #temporary log !!!!!!!!!!!!!
        orca_run(method_file, cluster_dir_path + log_file, workdir=cluster_dir_path, parallel=True)
    
parallel_wait()

In [ ]:
# check output && discard non-converging clusters
number_of_clusters = len(clusters)

for cluster in clusters:
    log_file = f'{output_path}{cluster}/{cluster}.out'
    with open(log_file) as infile:
        log = infile.read()
        if '****ORCA TERMINATED NORMALLY****' not in log:
            !{orca_job_check} {log_file}
            clusters.remove(cluster)

print(f'''{len(clusters)}/{number_of_clusters} successfully converged - unconverged are not considered
           in next steps.\n You can view logs at "{output_path}" directory''')

### 4.4 BP86 TZVP General gradient approximation method

In [ ]:
# specify BP86 method
method = '!BP86 def2-TZVP TightSCF Opt'

# input directory of .pdb representatives
input_dir = 'bp86svp/output/'

# output directory where converted .inp Orca 
# methods will be placed
output_dir = 'bp86tzvp/input/'

# number of CPUs to use
nprocs = 12

# specify spin (and charge)
spin = 1
# charge is specified in the first step 
# of 1. Molecule shape processing
# charge = XXX


for cluster in clusters:
    infile = f'{input_dir}{cluster}/{cluster}.xyz'
    outfile = f'{output_dir}{cluster}.inp'
    
    pdb2orca(infile, outfile, method, (charge,spin), nprocs=nprocs, xyz=True)

In [ ]:
# run the BP86 TZVP General gradient approximation method

input_path = 'bp86tzvp/input/'
output_path = 'bp86tzvp/output/'


for method_file in os.listdir(input_path):
    if '.inp' in method_file:
        log_file = method_file.replace('inp', 'out')
        cluster_dir = method_file.replace('.inp', '/')
        cluster_dir_path = output_path + cluster_dir

        if not os.path.exists(cluster_dir_path):
            os.mkdir(cluster_dir_path)    
        shutil.copy(input_path + method_file, cluster_dir_path)

        #temporary log !!!!!!!!!!!!!
        orca_run(method_file, cluster_dir_path + log_file, workdir=cluster_dir_path, parallel=True)
    
parallel_wait()

In [ ]:
# check output && discard non-converging clusters
number_of_clusters = len(clusters)

for cluster in clusters:
    log_file = f'{output_path}{cluster}/{cluster}.out'
    with open(log_file) as infile:
        log = infile.read()
        if '****ORCA TERMINATED NORMALLY****' not in log:
            !{orca_job_check} {log_file}
            clusters.remove(cluster)

print(f'''{len(clusters)}/{number_of_clusters} successfully converged - unconverged are not considered
           in next steps.\n You can view logs at "{output_path}" directory''')

In [ ]:
# extract final energies from output of Orca TZVP method
# note: energies are in hartree unit
orca_energies = []

for cluster in clusters:
    with open(f'bp86tzvp/output/{cluster}/{cluster}.out') as infile:
        for line in reversed(list(infile)):
            energy_list = re.findall(r'(FINAL SINGLE POINT ENERGY)( +)(-?\d+\.\d+)', line)
            if len(energy_list) > 0:
                orca_energies.append(float(energy_list[0][2]))
                break

In [ ]:
# convert from hartree to kJ/mol

CONVERSION_CONST = 2625.499638
min_energy = min(orca_energies)

energies_in_kJ = []
for energy in orca_energies:
    energies_in_kJ.append((energy-min_energy)*CONVERSION_CONST)

In [ ]:
#copy atoms from conformation before QM and combine it with orca optimised conformations

output_dir = "pdb_opt/"
input_dir = "bp86tzvp/output/outCluster"


atoms = []
with open("clustering/outClustersPDB/outCluster0.pdb", "r") as ifile:
    for line in ifile.readlines():
        if "ATOM" in line:
            atoms.append(line[:26])

for i in clusters_ids:
    hetatms = []
    !babel -ixyz {input_dir}{i}/outCluster{i}.xyz -opdb {output_dir}temp_cluster_{i}.pdb
    with open("{}temp_cluster_{}.pdb".format(output_dir, i), "r") as ifile:
        for line in ifile.readlines():
            if "HETATM" in line:
                hetatms.append(line[27:66])
    with open("pdb_opt/cluster{}.pdb".format(i), "w") as output_cluster:
        for j in range(len(atoms)):
            output_cluster.write(atoms[j])
            output_cluster.write(hetatms[j] + "\n")

!rm {output_dir}/temp_*

In [ ]:
input_dir = "pdb_opt/"
output_file = "clusters.pdb"

#concatinate optimised conformations to trajectory
with open(output_file, "w") as ofile:
    for i in clusters_ids:
        ofile.write("MODEL {}\n".format(str(i)))
        with open("{}cluster{}.pdb".format(input_dir, i), "r") as ifile:
            ofile.write(ifile.read())
            ofile.write("ENDMDL\n")

with open("plumed.dat", "w") as ifile:
    cvs = []
    num_atoms = sum(1 for line in open("{}cluster{}.pdb".format(input_dir, clusters_ids[0])))
    ifile.write("WHOLEMOLECULES ENTITY0=1-{}\n".format(str(num_atoms)))
    for i in range(0, len(torsions)):
        cvs.append("cv{}".format(i))
        ifile.write("TORSION ATOMS=")
        ifile.write(",".join(str(x) for x in torsions[i]))
        ifile.write(" LABEL={}\n".format(cvs[i]))
    ifile.write("PRINT ARG=")
    ifile.write(",".join(cvs))
    ifile.write(" STRIDE=1 FILE=DIHEDRALS")

#compute dihedrals
gmx_run(f"driver --plumed plumed.dat --mf_pdb {output_file}")

lines = []
with open("DIHEDRALS", "r") as ifile:
    for line in ifile.readlines():
        if "#" not in line:
            lines.append(line)
with open("DIHEDRALS", "w") as ofile:
    for line in lines:
        ofile.write(line)

In [ ]:
#perform minimisations and compute GAFF energy

cvs = [[] for _ in range(len(clusters_ids))]
with open("DIHEDRALS","r") as ifile:
    dihedrals = ifile.readlines()
    for i in range(len(dihedrals)):
        t_angles = dihedrals[i].split()
        for j in range(len(torsions)):
            cvs[i].append(float(t_angles[j+1])*(180/math.pi))

def generate_restraint(cluster_i):
    with open("MOL_GMX.top", "r") as ifile, open("gaff/cluster_{}/restrained.top".format(str(cluster_i)), "w") as ofile:
        for line in ifile.readlines():
            if line == "; Ligand position restraints\n":
                ofile.write("\n")
                ofile.write("[ dihedral_restraints ]\n")
                for j in range(len(torsions)):
                    ofile.write(" ".join(torsions[j]))
                    ofile.write("2 %3.1f 0 500\n" %cvs[i][j])
            ofile.write(line)

#select groups for energy evaluation
groups = "10"

for i in tqdm(clusters_ids):
    !mkdir -p -m 757 gaff/cluster_{i}
    shutil.copy("pdb_opt/cluster{}.pdb".format(i), "gaff/cluster_{}".format(i))
    shutil.copy("MOL_GMX.top", "gaff/cluster_{}".format(i))
    shutil.copy("em.mdp", "gaff/cluster_{}".format(i))
    shutil.copy("md.mdp", "gaff/cluster_{}".format(i))
    generate_restraint(i)
    gmx_run(f"editconf -f cluster{i}.pdb -box 3 3 3 -bt cubic -c -o box.gro", workdir=f"/gaff/cluster_{i}")
    gmx_run("grompp -f em -c box.gro -p restrained.top -o em1", workdir=f"/gaff/cluster_{i}")
    gmx_run("mdrun -ntomp 2 -s em1 -c after_em1 -g em1 -e em1 -o em1", workdir=f"/gaff/cluster_{i}")
    gmx_run("grompp -f md -c box.gro -p MOL_GMX.top -o rerun", workdir=f"/gaff/cluster_{i}")
    gmx_run("mdrun -ntomp 2 -s rerun -rerun em1 -c after_rerun -g rerun -e rerun -o rerun", workdir=f"/gaff/cluster_{i}")
    gmx_run("energy -f rerun.edr -o rerun.xvg", workdir=f"/gaff/cluster_{i}", groups=groups)

In [ ]:
#extract energies and from each energy value subtract minimal energy

energies_lst = []

for i in clusters_ids:
    with open("gaff/cluster_{}/rerun.xvg".format(i, "r")) as ifile:
        last_line = ifile.readlines()[-1]
        energies = last_line.split(" ")
        energies_lst.append(energies[len(energies) - 1].rstrip())

min_energy = min(energies_lst)
with open("gaff_energies.txt", "w") as ofile:
    for energy in energies_lst:
        ofile.write("{} \n".format((float(energy) - float(min_energy))))

## 5. Define correction of force field

In [ ]:
#subtract from orca energies computed in step 5, GAFF energies computed in step 6

with open("reference", "w") as ofile:
    with open("orca_energies.txt", "r") as orca_energies, open("gaff_energies.txt", "r") as gaff_energies:
        for orca_energy in orca_energies.readlines():
            for gaff_energy in gaff_energies.readlines():
                ofile.write("{} \n".format((float(orca_energy) - float(gaff_energy))))

In [ ]:
#write final corrections into output file along with representative conformations

input_dir = "pdb_opt/"

with open("reference.pdb", "w") as ofile, open("reference", "r") as ifile:
    final_energies = ifile.readlines()
    energy_index = 0
    for i in clusters_ids:
        ofile.write("REMARK X={}".format(final_energies[energy_index]))
        with open("{}cluster{}.pdb".format(input_dir, i), "r") as ifile1:
            ofile.writelines(ifile1.readlines())
        energy_index += 1

In [ ]:
x1 = []
y1 = []

with open("DIHEDRALS") as ifile:
    for line in ifile.readlines():
        split_values = line.split()
        x1.append(split_values[1])
        y1.append(split_values[2])

plot_landmarks(x, y, x1, y1).show()